In [17]:
####### USER INPUT #######
import pandas as pd

OpenBal = 29050    # type bal as at 1 Jan 2021
mydate = pd.to_datetime('2021-02-8')    # enter the date you want to forecast

In [18]:

def cat(mytext):
    
    # list of keywords, add as you find new ones
    WolliRent        = ['slick','carena','wolli income']
    WolliInterest    = ['LN REPAY 778047926','wolli expense','package annual']
    ThePondRent      = ['harcourts','hills Prestige','hillside','the pond income']
    ThePondInterest  = ['876891821','778047934','the pond expense','admiral']
    Grocery          = ['Coles','woolworths','woolworth','tongli','WW Metro','supermarket','iga','munch']
    Entertainment    = ['rebel','chemist','mcdonald','malatang','kmart','office','chatime','kogan','tea','indo'
                       ,'pondok','bread','subway','gong','chinese','kfc','mie','ciak','japanese','thai',
                       'saigon','bakery','racha','taste','food','card pocket money','sushi','domino','coffee',
                       'denmar','kurtosh','toastie','mazesoba','dopa','yummy','satang','daiso','bunning',
                       'reject shop','dough','kitkat','harper','woollahra ps','cafe','marigold','decathlon',
                       '7-eleven','merlin','golden summit','lamb','the growers','pancake','petok','pendopo',
                       'makan','grandbuy','churro','harry','big w','jim','martabak','emperors','pork','juice'
                       ,'chai','kopi','target','mart','butcher','sawah','water','aqua','sheng','crocodile'
                       ,'mumbai','indian','nandos','pho','curry','kitchen','Cook','Pastry']
    Salary           = ['salary']
    Utilities        = ['TPG','ENERGY','AGL','Netflix','OPTUS','Gas','Electricity','Telco','Prime']
    Insurance        = ['life','ins','medibank']
    Car              = ['Top One','AAMI','BPme','Etoll','Petrol']
    Rental           = ['Quay','DEFT RENT']
    Excluded         = ['Payment','2k']
    Discretionary    = ['Transfer To','Cardless Cash','Reid Cycle','Ikea',
                        'Good Guys','Harvey','JB Hi-Fi','CrazySales','cosmetic','baby','zara','H&M'
                        ,'Cotton On','uniqlo']
    # combine all the categories into one list called collection
    collection      = [WolliRent,WolliInterest,ThePondRent,ThePondInterest,Grocery,Entertainment,Salary,
                       Utilities,Insurance,Car,Rental,Excluded,Discretionary]


    for i in range(len(collection)):
        for k in range(len(collection[i])):
            if collection[i][k].lower() in mytext.lower():
                if i == 0:
                    cat = 'Wolli - Income'
                elif i == 1:
                    cat = 'Wolli - Expense'
                elif i == 2:
                    cat = 'The Pond - Income'
                elif i == 3:
                    cat = 'The Pond - Expense'
                elif i == 4:
                    cat = 'Grocery'
                elif i == 5:
                    cat = 'Food & Entertainment'
                elif i == 6:
                    cat = 'Salary'
                elif i == 7:
                    cat = 'Utilities'   
                elif i == 8:
                    cat = 'Insurance'   
                elif i == 9:
                    cat = 'Car'   
                elif i == 10:
                    cat = 'Rental'
                elif i == 11:
                    cat = 'Exclude'
                elif i == 12:
                    cat = 'Discretionary'   
                return cat
            
   

In [30]:
import pandas as pd
import numpy as np

EndDate = '2021-12-31'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq Offfset
mylist = [['The Pond Income  - Rent'       ,650,'2021-01-5','','7D',0],
          ['The Pond Expense - WBC'        ,-889,'2021-01-6','','7D',0],
          ['The Pond Expense - CBA1'       ,-161,'2020-12-7','','MS',6],
          ['The Pond Expense - CBA2'       ,-307,'2020-12-9','','MS',9],
          ['The Pond Income  - Tree'       ,-400,'2021-01-15','2021-01-15','7D',0],    # adhoc just put end end date
          ['The Pond Income  - Water Pump' ,-1450,'2021-01-15','2021-01-15','7D',0],   # adhoc just put end e
          ['The Pond Income  - Water'      ,-147,'2021-01-1','','3MS',0],
          ['The Pond Income  - Council'    ,-400,'2021-02-28','','3MS',27],
          
          
          ['Wolli Income  - Rent'      ,450,'2021-02-06','','7D',0],
          ['Wolli Expense - Strata'    ,-1183,'2021-03-26','','3MS',25],
          ['Wolli Expense - Water'     ,-144,'2021-04-20','','3MS',19],
          ['Wolli Expense - Council'   ,-348,'2021-02-28','','3MS',27],
          ['Wolli Expense - CBA'       ,-311,'2021-01-6','','7D',0],
          ['Wolli Expense - Insurance' ,-35.59,'2021-01-15','','MS',15],
          
          
          ['Salary'              ,3413.17,'2021-01-13','','14D',0],
          ['Tax Return'          ,7000,'2021-07-30','2021-07-30','7D',0],
          ['Rent Quay St'        ,-700,'2021-01-4','','7D',0],
          ['Parents'             ,-100,'2021-01-4','','7D',0],
          ['Var Food & Ent'      ,-375,'2021-01-4','','7D',0],
          
          ['AAMI Car'            ,-94.2,'2020-12-18','','MS',17],
          ['Car Petrol'          ,-30,'2021-01-01','','7D',0],
          ['TPG'                 ,-59.99,'2020-12-2','','MS',1],
          ['Gas'                 ,-191,'2020-12-5','','MS',4],
          ['Electricity'         ,-238,'2021-02-25','','3MS',24],  #MS means beg of month, M means end of month
          ['Optus'               ,-60,'2020-12-21','','MS',20],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    if mylist[i][3] == '':
        ToDate = EndDate
        #print(ToDate,mylist[i][0])
    else:
        ToDate = mylist[i][3]
        #print(ToDate,mylist[i][0])
    date = pd.date_range(mylist[i][2],ToDate,freq=mylist[i][4]) + pd.DateOffset(days=mylist[i][5])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date'],dayfirst=True).dt.to_period('M')
df['Cat']= df.apply(lambda x:cat(x['Description']),axis=1)
#df['YTD'] = df.groupby(['Description'])['Amount'].transform(pd.Series.cumsum)


# create forecast table for specific period
tbl_forecast = df[df['Date']<=mydate].sort_values('Cat')
forecast = tbl_forecast[['Cat','Amount']].groupby(['Cat']).sum().reset_index()


########### download and run actual from CBA statements etc   ##########


import pandas as pd
cba = pd.read_csv(r'/Users/ronny/Downloads/CSVData-14.csv', header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Description']
cba['Date'] = pd.to_datetime(cba['Date'])


cba['Cat'] = cba.apply(lambda x:cat(x['Description']),axis=1)
cba['Cat'].fillna('Other',inplace=True)
# add Period
cba['Period'] = pd.to_datetime(cba['Date'],dayfirst=True).dt.to_period('M')
tbl_actual = cba

#cba.to_csv(r'C:\Users\m118954\mydsbook\Data\CSVData2.csv')

actual = cba.groupby('Cat').sum().reset_index()


budget = pd.merge(forecast,actual,on=['Cat'],how='left')
budget['Var'] = budget['Amount_y'] - budget['Amount_x']
budget.columns=['Category','Forecast','Actual','Variance']



In [33]:
pd.options.display.float_format = '{:,.2f}'.format
from IPython.display import display

import datetime 


bal = df[df['Date']<=mydate]['Amount'].sum()
bal_actual = cba[cba['Date']<=mydate]['Amount'].sum()   # check CBA Balance Later, does not reconcile
df1 = df[df['Date']<=mydate].groupby(['Description']).sum()

display ('Your Forecast Income at the end of ' + str(mydate) + ' is ' + str(bal))
display ('Your Forecast Balance at the end of ' + str(mydate) + ' is ' + str(bal+OpenBal))

display ('Your Actual Income at the end of ' + str(mydate) + ' is ' + str(bal_actual))
display ('Your Actual Balance at the end of ' + str(mydate) + ' is ' + str(OpenBal+bal_actual))

df1 = df1.reset_index()
df1['Cat']= df1.apply(lambda x:cat(x['Description']),axis=1)
df1 = df1[['Cat','Description','Amount']].sort_values('Cat')



print ('### This is how you are tracking against forecast in your CBA Offset ####')
display(budget)


print ('\n\n #### Report 2 - Granular Forecast ####')
display(df1)

'Your Forecast Income at the end of 2021-02-08 00:00:00 is -5985.839999999999'

'Your Forecast Balance at the end of 2021-02-08 00:00:00 is 23064.16'

'Your Actual Income at the end of 2021-02-08 00:00:00 is -1987.6399999999992'

'Your Actual Balance at the end of 2021-02-08 00:00:00 is 27062.36'

### This is how you are tracking against forecast in your CBA Offset ####


,Category,Forecast,Actual,Variance
0,Car,-274.20,-188.40,85.80
1,Food & Entertainment,"-2,250.00","-1,795.22",454.78
2,Rental,"-4,200.00","-3,500.00",700.00
3,Salary,"6,826.34","3,669.54","-3,156.80"
4,The Pond - Expense,"-5,074.00","-4,028.37","1,045.63"
5,The Pond - Income,"1,253.00","1,650.40",397.40
6,Utilities,-561.98,-119.99,441.99
7,Wolli - Expense,"-1,555.00","-1,244.00",311.00
8,Wolli - Income,450.00,nan,nan




 #### Report 2 - Granular Forecast ####


,Cat,Description,Amount
0,Car,AAMI Car,-94.20
1,Car,Car Petrol,-180.00
15,Food & Entertainment,Var Food & Ent,"-2,250.00"
5,Rental,Rent Quay St,"-4,200.00"
6,Salary,Salary,"6,826.34"
10,The Pond - Expense,The Pond Expense - WBC,"-4,445.00"
8,The Pond - Expense,The Pond Expense - CBA1,-322.00
9,The Pond - Expense,The Pond Expense - CBA2,-307.00
11,The Pond - Income,The Pond Income - Rent,"3,250.00"
12,The Pond - Income,The Pond Income - Tree,-400.00


# Used Below query to see details of discrepancy ac vs forecast

In [9]:
from IPython.display import display


check = 'Salary'
chk_forecast = tbl_forecast[tbl_forecast['Cat']==check]
chk_actual =  tbl_actual[tbl_actual['Cat']==check]

print ('#### Forecast Details ####')
display (chk_forecast)

print ('#### Actual Details ####')
display (chk_actual)


#### Forecast Details ####


,Date,Description,Amount,Period,Cat
1,2021-01-27,Salary,"3,413.17",2021-01,Salary
2,2021-02-10,Salary,"3,413.17",2021-02,Salary
0,2021-01-13,Salary,"3,413.17",2021-01,Salary


#### Actual Details ####


,Date,Amount,Description,Cat,Period
7,03/02/2021,512.74,Transfer From RONNY CHRISTIANTO salary,Salary,2021-02
15,28/01/2021,"3,156.80",Direct Credit 421520 RONNY CHRISTIANT Ronny Sa...,Salary,2021-01


In [ ]:
import pandas as pd

#import pandas as pd
#import numpy as np
#import os
#import glob

# pick up all offset files
#os.chdir("/Users/ronny/Downloads")
#extension = 'csv'
#offset_file = [i for i in glob.glob('*.{}'.format(extension))]




card = pd.read_csv(r'/Users/ronny/Downloads/CSVData-13.csv', header=None,usecols=[0,1,2])
card.columns=['Date','Amount','Description']
card['Cat'] = card.apply(lambda x:cat(x['Description']),axis=1)
card['Cat'].fillna('Other',inplace=True)
card['Period'] = pd.to_datetime(card['Date'],dayfirst=True).dt.to_period('M')

In [ ]:
pd.pivot_table(card, values='Amount', index=['Cat'],
                    columns=['Period'], aggfunc=np.sum)

In [ ]:
card[(card['Period']=='2021-01') & (card['Cat']=='Other')].head(60)

In [ ]:
card[card['Cat']=='Other'].head(60)

In [ ]:
card.head()

In [ ]:
word_list = card['Description'].tolist()
word_counter = {}

for word in word_list:
    if word in word_counter:
         word_counter[word] += 1
    else:
         word_counter[word] = 1

popular_words = sorted(word_counter, key = word_counter.get, reverse = True)

display ('top 3 popular words are ' + str(popular_words[:3]))


for i in popular_words:
    print (i + ' ' + str(word_counter.get(i)))

In [13]:
cba['Amount'].sum()

-6230.57